# Pictures acquisition

In [ ]:
import cv2 as cv
import os
import re
import IPython.display as ipd

In [ ]:
import utils_2d

OpenCV fornisce una serie di modelli pre-addestrati per la detection di specifiche classi, basati sul metodo di Viola e Jones:<br/>
https://docs.opencv.org/3.4/db/d28/tutorial_cascade_classifier.html<br/>
https://docs.opencv.org/3.4/dc/d88/tutorial_traincascade.html

Nel caso di installazione basata su *conda*, questi modelli possono essere trovati in:<br/>
``.conda/envs/<ambiente>/Lib/site-packages/cv2/data/``

Nel caso di installazione basata su *virtualenv*, questi modelli possono essere trovati in:<br/>
``<ambiente>/lib/python<X>/site-packages/cv2/data/``

In [ ]:
# Caricamento modello per il rilevamento di volti frontali
model_filename = 'C:/Users/gianc/Anaconda3/envs/dsim/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml'
face_detector = cv.CascadeClassifier(model_filename)

`cv.CascadeClassifier.detectMultiScale(image[, scaleFactor[, minNeighbors[, flags[, minSize[, maxSize]]]]]`

Parametri aggiuntivi:

- https://docs.opencv.org/2.4/modules/objdetect/doc/cascade_classification.html
- https://docs.opencv.org/2.4/modules/core/doc/drawing_functions.html
- https://docs.opencv.org/3.4.7/dc/da5/tutorial_py_drawing_functions.html

In [ ]:
# this is the only time this function is used, so I would not add it to utils_2d
# this is also the only time the face detector from opencv is used
def crop_picture(img, more_space=30):
    m = more_space # some additional space around the cropped face
    if img is not None:
        faces = face_detector.detectMultiScale(img[:,:,1]) # (img_gray)
        if len(faces) > 0:
            (x,y,w,h) = faces[0]
            img = img[(y-m):(y+h+m), (x-m):(x+w+m), :]
    return img[:,::-1]

In [ ]:
def save_pictures(name, basedir, file_format = "png", crop=True, more_space = 50):
    
    name_basedir = basedir + "/" + name + "/"
    if not os.path.isdir(name_basedir):
        print(f"Created new directory {name_basedir}")
        os.makedirs(name_basedir)

    current_images = [img for img in os.listdir(name_basedir) if img.endswith(file_format)]
    
    filename_format = f"{basedir}/{name}/{name}_"+"{}"+f".{file_format}"
    
    i = utils_2d.find_last_filename_id(current_images)
    
    cap = cv.VideoCapture(0)
    while(True):
        
        r, frame = cap.read()
        if crop:
            frame = crop_picture(frame, more_space)
            
        try:
            cv.imshow('Video', frame)
        except Exception as e:
            print(e)
            pass

        # Save is 's' is pressed
        if cv.waitKey(1) & 0xFF == ord('s'):
            
            # If crop is set to true, to make sure that the image has been cropped we check if it is a square
            if frame.shape[0] == frame.shape[1] or (not crop):
                
                i += 1
                filename = filename_format.format(i)
                cv.imwrite(filename, frame)
                ipd.clear_output(wait=True)
                print(f"Saved {filename}".replace("//", "/"))
                
        # exit if 'q' is pressed
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv.destroyAllWindows()

In case of errors, make sure the webcam is not used by other processes/notebooks.

Hold "s" down to save pictures quickly:

In [ ]:
name = "gian"
basedir = "./pictures_new/"
save_pictures(name, basedir, crop = True, more_space = 30)

It works, but sometimes you need to spam "q" to quit.